# Lab 1: Access Control with Azure AI Gateway

Test different authentication methods through APIM.

## Learning Objectives
- Understand APIM authentication policies
- Test OAuth 2.0 / JWT token validation
- Compare API Key vs Token-based auth
- Configure dual authentication

## Prerequisites
- Azure CLI authenticated (`az login`)
- Resources deployed via main notebook
- APIM with access control policies configured

**Duration:** ~10 minutes  
**Difficulty:** Beginner

---

In [ ]:
# Initialize environment
import sys
sys.path.append('..')
from quick_start.shared_init import quick_init, get_azure_openai_client
import requests
import json
import os

config = quick_init()

# Get APIM endpoint
apim_gateway_url = config['env']['APIM_GATEWAY_URL']
print(f"\n🔗 APIM Gateway: {apim_gateway_url}")

## Test 1: No Authentication (Expected: 401 Unauthorized)

This should fail - APIM requires authentication.

In [ ]:
# Test without any authentication
endpoint = f"{apim_gateway_url}/deployments/gpt-4o-mini/chat/completions?api-version=2024-10-21"

response = requests.post(
    endpoint,
    headers={"Content-Type": "application/json"},
    json={
        "messages": [{"role": "user", "content": "Hello"}],
        "max_tokens": 10
    }
)

print(f"Status: {response.status_code}")
if response.status_code == 401:
    print("✅ Expected behavior: Access denied without authentication")
else:
    print(f"Response: {response.text[:200]}")

## Test 2: API Key Authentication

Using APIM subscription key (Ocp-Apim-Subscription-Key header).

In [ ]:
# Get API key from environment
api_key = os.getenv('APIM_SUBSCRIPTION_KEY')

if not api_key:
    print("⚠️ APIM_SUBSCRIPTION_KEY not set in master-lab.env")
else:
    response = requests.post(
        endpoint,
        headers={
            "Content-Type": "application/json",
            "Ocp-Apim-Subscription-Key": api_key
        },
        json={
            "messages": [{"role": "user", "content": "Hello"}],
            "max_tokens": 10
        }
    )

    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        result = response.json()
        print(f"✅ Success with API Key")
        print(f"Response: {result['choices'][0]['message']['content']}")
    else:
        print(f"❌ Failed: {response.text[:200]}")

## Test 3: OAuth 2.0 / JWT Token Authentication

Using Azure AD token (Bearer authentication).

In [ ]:
from azure.identity import AzureCliCredential

# Get Azure AD token
credential = AzureCliCredential()
token = credential.get_token("https://cognitiveservices.azure.com/.default")

response = requests.post(
    endpoint,
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token.token}"
    },
    json={
        "messages": [{"role": "user", "content": "Hello with JWT"}],
        "max_tokens": 15
    }
)

print(f"Status: {response.status_code}")
if response.status_code == 200:
    result = response.json()
    print(f"✅ Success with JWT Token")
    print(f"Response: {result['choices'][0]['message']['content']}")
else:
    print(f"❌ Failed: {response.text[:200]}")

## Test 4: Azure OpenAI Client (Easiest Method)

The shared_init module already handles authentication for you.

In [ ]:
# Use the helper function - authentication is automatic
client = get_azure_openai_client()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "Explain Azure AD authentication in one sentence."}
    ],
    max_tokens=50
)

print("✅ Success using AzureOpenAI client")
print(f"\nResponse:\n{response.choices[0].message.content}")

## Summary

| Method | Status | Use Case |
|--------|--------|----------|
| No Auth | ❌ 401 | - |
| API Key | ✅ 200 | Simple apps, testing |
| JWT Token | ✅ 200 | Enterprise apps, Zero Trust |
| AzureOpenAI Client | ✅ 200 | Development (easiest) |

## What You Learned

1. **APIM enforces authentication** - No anonymous access allowed
2. **Multiple auth methods** - API keys for simplicity, JWT tokens for security
3. **Azure CLI credentials** - Easiest method for development
4. **Production recommendation** - Use managed identity or JWT validation

## Next Steps

- Configure APIM policies for your authentication requirements
- Set up Azure AD app registration for production JWT validation
- Implement role-based access control (RBAC)
- Move to managed identity for Azure-hosted apps

**Next Lab:** `02-semantic-caching.ipynb` - Implement Redis caching